#IMDB영화평 감성분석

- pipleline
- TfidVectorizer + LogisticRegression

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
up = files.upload()


Saving labeledTrainData.tsv to labeledTrainData.tsv


In [2]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
# 텍스트 전처리
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

# 구둣점, 숫제는 제거, 영문자가 아닌 글자는 공백으로 변환 (re를 쓰지 않고도 데이터프레임의 str method는 정규표현식을 지원함)
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()
df.review[100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


'There is a uk edition to this show which is rather less extravagant than the US version  The person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got  The US version of this show is everything that reality TV shouldn t be  Instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt  I do not know if this show is trying to show what a lousy welfare system exists in the US or if you beg hard enough you will receive  The rather vulgar product placement that takes place  particularly by Sears  is also uncalled for  Rsther than turning one family in a deprived area into potential millionaires  it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home  build something for the whole community       perhaps a place where diy and power tools can be borrowed and returned along with build

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df.review, df.sentiment, stratify=df.sentiment, random_state=2022)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

Pipeline : TfidVectorizer + LogisticRegression

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [8]:
tvect = TfidfVectorizer(ngram_range=(1,2), stop_words = 'english')
lrc = LogisticRegression(random_state=2022)

pipeline = Pipeline([
                    ('TVECT', tvect), ('LR', lrc)
])

In [9]:
#학습
%time pipeline.fit(X_train,y_train)

CPU times: user 36.6 s, sys: 14.8 s, total: 51.4 s
Wall time: 36.4 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LR', LogisticRegression(random_state=2022))])

In [10]:
pipeline.score(X_test,y_test)

0.87472

In [12]:
#GridSearchCV를 이용하여 최적 파라미터 찾기
from sklearn.model_selection import GridSearchCV
params = {
    'TVECT__max_df':[100, 500],
    'LR__C':[1,10]
}

In [14]:
grid_pipe = GridSearchCV(pipeline, params, scoring = 'accuracy', cv=3, n_jobs=1)
%time grid_pipe.fit(X_train, y_train )

CPU times: user 4min 33s, sys: 1min 33s, total: 6min 6s
Wall time: 4min 26s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=1,
             param_grid={'LR__C': [1, 10], 'TVECT__max_df': [100, 500]},
             scoring='accuracy')

In [15]:
grid_pipe.best_params_

{'LR__C': 10, 'TVECT__max_df': 500}

In [16]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.87552

In [17]:
import joblib
joblib.dump(grid_pipe.best_estimator_, 'imdb_tvect_lr.pkl')

['imdb_tvect_lr.pkl']

In [18]:
best_pipe = joblib.load('imdb_tvect_lr.pkl')

In [19]:
#실제 데이터 적용

review = '''Epic film about a Mafia family called the Corleones. It takes place from the 1940s to 1950s. The family is overseen by Don Corleone (Marlon Brando). His sons help their father...except for his youngest Michael (Al Pacino). He wants no part of the family business...but he might have no choice.

A fantastic film. It runs three hours but you're never bored. It's wonderfully directed by Francis Ford Coppola. It also is one of those rare films with a perfect cast--no one is bad! It's also fun to see Pacino, James Caan, Robert Duvall and Diane Keaton so young and full of life. Many of the lines ("I'll make him an offer he can't refuse") have become legendary. Also there is some violence which was considered extreme in 1972 but is pretty tame today. Only a character getting shoot at a toll booth is pretty over the top. My favorite scene has to be at the end when Keaton realizes what she's married into. The look on her face is perfect.

This was a HUGE hit at the box office. It was loved by critics and the public alike. It was (rightfully) called the Gone With the Wind of the 1970s. A masterpiece and well worth seeing.'''



In [24]:
# 텍스트 전처리
# <br /> 태그는 공백으로 변환
review = review.replace('<br />',' ')

# 구둣점, 숫제는 제거, 영문자가 아닌 글자는 공백으로 변환 (re를 쓰지 않고도 데이터프레임의 str method는 정규표현식을 지원함)
review = review.replace('[^A-Za-z]',' ').strip()
review

'Epic film about a Mafia family called the Corleones. It takes place from the 1940s to 1950s. The family is overseen by Don Corleone (Marlon Brando). His sons help their father...except for his youngest Michael (Al Pacino). He wants no part of the family business...but he might have no choice.\n\nA fantastic film. It runs three hours but you\'re never bored. It\'s wonderfully directed by Francis Ford Coppola. It also is one of those rare films with a perfect cast--no one is bad! It\'s also fun to see Pacino, James Caan, Robert Duvall and Diane Keaton so young and full of life. Many of the lines ("I\'ll make him an offer he can\'t refuse") have become legendary. Also there is some violence which was considered extreme in 1972 but is pretty tame today. Only a character getting shoot at a toll booth is pretty over the top. My favorite scene has to be at the end when Keaton realizes what she\'s married into. The look on her face is perfect.\n\nThis was a HUGE hit at the box office. It was 

In [26]:
best_pipe.predict([review])


array([1])